In [1]:
import sys

print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

Python version
3.8.19 (default, Mar 20 2024, 15:27:52) 
[Clang 14.0.6 ]
Version info.
sys.version_info(major=3, minor=8, micro=19, releaselevel='final', serial=0)


In [ ]:
# Notes: 
# #torch
#!pip install transformers

# Plate2Recipe Project


## Data Collection and Preprocessing


- Data loading: *use food-101.tar.gz for CNN and full_dataset.csv for NLP* (https://drive.google.com/drive/folders/1ui_zS11_ENZTCNLUsgg_UwAYr-ZaLbac)
- Data cleaning
- Data augmentation

In [6]:
# CNN - Load and Transform Data

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

root_dir = './data/'

train_dataset = datasets.Food101(root=root_dir, split='train', transform=train_transforms, download=True)
valid_dataset = datasets.Food101(root=root_dir, split='test', transform=valid_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

Using downloaded and verified file: ./data/food-101.tar.gz
Extracting ./data/food-101.tar.gz to ./data/


In [65]:
# NLP - Load data

import pandas as pd

recipes = pd.read_csv('./data/full_dataset.csv', encoding='UTF-8')
recipes = recipes.drop(recipes.columns[0], axis=1)
recipes.head(5)

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


## Model Development
- CNN for Image Processing
- NLP for Recipe Generation


### NLP Model
- Assume list of ingredients is provided, we can use this to generate the recipe. 
- Match input list of ingredients with the ingredients in the dataset and generate the recipe
- Approaches:
        -- Best Matching using Cosine Similarity: The chosen recipe will be the one with the highest similarity score
        -- GPT2 model
        -- ...

In [60]:
# Assume 'user_ingredients' list is already defined
user_ingredients = ['chicken', 'onion', 'pepper']

- #### Using TF-IDF for vectorization & cosine similarity for best matching

In [61]:
# using TF-IDF for vectorization and cosine similarity for matching

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Preprocess data:
def preprocess(ingredients):
    return [' '.join(eval(ing)).lower().replace('[^a-z\s]', '') for ing in ingredients]

# VECTORIZE
vectorizer = TfidfVectorizer()

# Vectorize ingredients: 
recipe_ingredients = preprocess(recipes['NER'])
recipe_tfidf = vectorizer.fit_transform(recipe_ingredients)

# Vectorize user ingredients:
user_ingredients_string = ' '.join(user_ingredients).lower()
user_tfidf = vectorizer.transform([user_ingredients_string])


# COMPUTE COSINE SIMILARITY
similarity_scores = cosine_similarity(user_tfidf, recipe_tfidf)

# Select the best matching recipe
best_match_index = similarity_scores.argmax()
best_recipe = recipes.iloc[best_match_index]

print("Best Matching Recipe:", best_recipe['title'])
print("Ingredients:", best_recipe['ingredients'])
print("Directions:", best_recipe['directions'])

Best Matching Recipe: My Chopped Chicken Liver
Ingredients: ["1 lb chicken liver", "3 tablespoons chicken fat (SHMALTZ)", "1 onion, diced", "3 hard-boiled eggs, chopped", "salt", "pepper", "onion powder", "garlic powder"]
Directions: ["In microwave, cook chicken livers and fat until done, about 5 minutes (use a pie dish).", "Remove livers with a slotted spoon, add onions and microwave until cooked, about 3 minutes.", "Meanwhile hard boil eggs in saucepan until done.", "Add livers, fat and onions to food processor; pulse until all ingredients are chopped fine.", "Mash eggs and add seasoning.", "Add liver mixture to egg mixture.", "Serve chopped liver on greens, surrounded by tomatoes, and lettuce.", "Serve with toast points or crackers."]


- #### Using GPT-2: 
This approach combines traditional NLP techniques for ingredient matching with advanced language generation capabilities of GPT-2


In [64]:
# Use GPT-2 to generate a Recipe

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load pre-trained model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Input Text to GPT-2: provide context to GPT-2 for generating the recipe. The format & content can be adjusted based on how we want GPT-2 to expand the recipe information.
text = f"Ingredients: {', '.join(user_ingredients)}; Recipe: {best_recipe['title']} - Directions:"

# Encode the text input
indexed_tokens = tokenizer.encode(text, add_special_tokens=True)
tokens_tensor = torch.tensor([indexed_tokens])

# Generate a text using the model
model.eval()
with torch.no_grad():
    outputs = model.generate(tokens_tensor, max_length=300, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: chicken, onion, pepper; Recipe: My Chopped Chicken Liver - Directions: Preheat oven to 350 degrees F. Line a baking sheet with parchment paper and set aside. In a large bowl, whisk together the olive oil, salt, and pepper. Add the chicken mixture to the dry ingredients and mix well. Pour the mixture into the prepared baking dish and bake for 20-25 minutes, or until golden brown. Remove from the oven and allow to cool completely before serving.


## Training
- Model compilation
- Model training
- Hyperparameter tuning

## Evaluation
- Model evaluation metrics
- Visualization of results

## Conclusion
- Summary of findings
- Future work